In [1]:
import torch
import random
import os
import pandas as pd
import torch_scatter
import torch.nn as nn
from torch.nn import Linear, Sequential, LayerNorm, ReLU
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.data import DataLoader
from torch_geometric.data.storage import GlobalStorage
import numpy as np
import time
import torch.optim as optim
from tqdm import trange
import copy
import matplotlib.pyplot as plt
import h5py
#import tensorflow.compat.v1 as tf
import functools
import json
from torch_geometric.data import Data
import enum

In [3]:
root_dir = os.getcwd()
dataset_dir = os.path.join(root_dir, 'data')
checkpoint_dir = os.path.join(root_dir, 'best_models')
postprocess_dir = os.path.join(root_dir, 'animations')

file_path = os.path.join(dataset_dir,'meshgraphnets_miniset5traj_vis.pt')
torch.serialization.add_safe_globals([GlobalStorage])
dataset_timesteps = torch.load(file_path, weights_only=False)  # full list (likely)
dataset = torch.load(file_path)[:1]                            # just first element

print(type(dataset_timesteps), len(dataset_timesteps))
print(type(dataset), len(dataset))


<class 'list'> 2995
<class 'list'> 1


In [4]:
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd() / "src"))  # add ./src to the import path

In [5]:
from dataset_utils import graph_basic_stats, dataset_basic_stats

# Basic stats on the first graph only (matches your 'dataset' slice)
g0_stats = graph_basic_stats(dataset[0])
g0_stats

{'num_nodes': 1923,
 'num_edges': 11070,
 'x_shape': [1923, 11],
 'edge_attr_shape': [11070, 3],
 'y_shape': [1923, 2],
 'x_has_nan': False,
 'edge_has_nan': False,
 'y_has_nan': False,
 'deg_mean': 5.7566304206848145,
 'deg_min': 2,
 'deg_max': 8,
 'v_x_mean': 1.3118444681167603,
 'v_x_std': 0.5271742343902588,
 'v_y_mean': 0.0006944937049411237,
 'v_y_std': 0.22921890020370483,
 'edge_len_mean': 0.019826507195830345,
 'edge_len_min': 0.002250395016744733,
 'edge_len_max': 0.04720745235681534,
 'a_x_mean': -4.79417085647583,
 'a_x_std': 13.742134094238281,
 'a_y_mean': 0.02090461179614067,
 'a_y_std': 5.087349891662598,
 'node_type_counts': [1689, 0, 0, 0, 17, 17, 200, 0, 0]}

In [6]:
# If dataset_timesteps is the full list, get overall counts
ds_stats = dataset_basic_stats(dataset_timesteps)
ds_stats


{'num_graphs': 2995,
 'total_nodes': 5531765,
 'total_edges': 31781742,
 'node_type_counts_total': [4827341, 0, 0, 0, 52113, 50915, 601396, 0, 0],
 'first_graph_stats': {'num_nodes': 1923,
  'num_edges': 11070,
  'x_shape': [1923, 11],
  'edge_attr_shape': [11070, 3],
  'y_shape': [1923, 2],
  'x_has_nan': False,
  'edge_has_nan': False,
  'y_has_nan': False,
  'deg_mean': 5.7566304206848145,
  'deg_min': 2,
  'deg_max': 8,
  'v_x_mean': 1.3118444681167603,
  'v_x_std': 0.5271742343902588,
  'v_y_mean': 0.0006944937049411237,
  'v_y_std': 0.22921890020370483,
  'edge_len_mean': 0.019826507195830345,
  'edge_len_min': 0.002250395016744733,
  'edge_len_max': 0.04720745235681534,
  'a_x_mean': -4.79417085647583,
  'a_x_std': 13.742134094238281,
  'a_y_mean': 0.02090461179614067,
  'a_y_std': 5.087349891662598,
  'node_type_counts': [1689, 0, 0, 0, 17, 17, 200, 0, 0]},
 'last_graph_stats': {'num_nodes': 1802,
  'num_edges': 10344,
  'x_shape': [1802, 11],
  'edge_attr_shape': [10344, 3],
 